Name: Aviral Jain | 
Roll No: 18AG3AI08 | 
BTP Guide: Prof. Rajendra Machavaram

# Application of Convolutional Neural Networks for Wheat Head Detection

Firstly we set up the required environment experiment and install necessary libraries and dependencies. 

In [ ]:
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install pycocotools 
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py

In [ ]:
import torch
print(torch.__version__)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import albumentations as A
from albumentations.pytorch import ToTensorV2 
import torchvision
from torchvision import datasets,transforms
from tqdm import tqdm
import cv2
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
from PIL import Image
import os
import torch.nn.functional as F
import ast

In [ ]:
# configure the hyperparameters here
LR = 1e-4
SPLIT = 0.2
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 4
EPOCHS = 2
DATAPATH = '../input/global-wheat-detection'

In [ ]:
!ls '../input/global-wheat-detection'

In [ ]:
df = pd.read_csv(DATAPATH + '/train.csv')
df.bbox = df.bbox.apply(ast.literal_eval)

In [ ]:
df = df.groupby("image_id")["bbox"].apply(list).reset_index(name="bboxes")

In [ ]:
def train_test_split(dataFrame,split):
    len_tot = len(dataFrame)
    val_len = int(split*len_tot)
    train_len = len_tot-val_len
    train_data,val_data = dataFrame.iloc[:train_len][:],dataFrame.iloc[train_len:][:]
    return train_data,val_data

In [ ]:
train_data_df,val_data_df = train_test_split(df,SPLIT)

In [ ]:
train_data_df

In [ ]:
class WheatDataset(Dataset):     #Create a dataset class which reads the images and bounding boxes and passes them into the dataloader in the correct format
    def __init__(self,data,root_dir,transform=None,train=True):
        self.data = data
        self.root_dir = root_dir
        self.image_names = self.data.image_id.values
        self.bboxes = self.data.bboxes.values
        self.transform = transform
        self.isTrain = train
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,index):
#         print(self.image_names)
#         print(self.bboxes)
        img_path = os.path.join(self.root_dir,self.image_names[index]+".jpg")
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        bboxes = torch.tensor(self.bboxes[index],dtype=torch.float64)
#         print(bboxes)
        """
            As per the docs of torchvision
            we need bboxes in format (xmin,ymin,xmax,ymax)
            Currently we have them in format (xmin,ymin,width,height)
        """
        bboxes[:,2] = bboxes[:,0]+bboxes[:,2]
        bboxes[:,3] = bboxes[:,1]+bboxes[:,3]
#         print(image.size,type(image))
        """
            we need to return image and a target dictionary
            target:
                boxes,labels,image_id,area,iscrowd
        """
        area = (bboxes[:,3]-bboxes[:,1])*(bboxes[:,2]-bboxes[:,0])
        area = torch.as_tensor(area,dtype=torch.float32)
        
        # there is only one class
        labels = torch.ones((len(bboxes),),dtype=torch.int64)
        
        # suppose all instances are not crowded
        iscrowd = torch.zeros((len(bboxes),),dtype=torch.int64)
        
        target = {}
        target['boxes'] = bboxes
        target['labels']= labels
        target['image_id'] = torch.tensor([index])
        target["area"] = area
        target['iscrowd'] = iscrowd
        
        if self.transform is not None:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transform(**sample)
            image = sample['image']
            
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)
            
        return image,target

In [ ]:
#Apply necessary transformations

train_transform = A.Compose([
    A.Flip(0.5),
    ToTensorV2(p=1.0)
],bbox_params = {'format':"pascal_voc",'label_fields': ['labels']})

val_transform = A.Compose([
      ToTensorV2(p=1.0)
],bbox_params = {'format':"pascal_voc","label_fields":['labels']})

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
train_data = WheatDataset(train_data_df,DATAPATH+"/train",transform=train_transform)
valid_data = WheatDataset(val_data_df,DATAPATH+"/train",transform=val_transform)

In [ ]:
image,target = train_data.__getitem__(0)
print(image.shape)

plt.imshow(image.permute(1, 2, 0))

Import the Faster R-CNN Model

In [ ]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
#We use the pretrained Faster-RCNN so that our training is faster
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 2
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes)

In [ ]:
class Averager:
    def __init__(self):
        self.current_total=0.0
        self.iterations = 0.0
    def send(self,value):
        self.current_total+=value
        self.iterations+=1
    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0*self.current_total/self.iterations
    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

Create torch based dataloader so that training is accelerated by GPU

In [ ]:
train_dataloader = DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True,collate_fn=collate_fn)
val_dataloader = DataLoader(valid_data,batch_size=BATCH_SIZE,shuffle=False,collate_fn=collate_fn)

print(len(train_dataloader))
print(len(val_dataloader))

In [ ]:
train_loss = []
val_loss = []
model = model.to(DEVICE)
params =[p for p in model.parameters() if p.requires_grad]
#We compile the model using Adaptive Momentum Optimizer
optimizer = optim.Adam(params,lr=LR)
loss_hist = Averager()
itr = 1
lr_scheduler=None

Start training

In [ ]:
from engine import train_one_epoch, evaluate

loss_hist = Averager()
itr = 1
model = model.float()
for epoch in range(EPOCHS):
    loss_hist.reset()
    
    for images, targets in train_dataloader:
        
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        print(losses.dtype)
        loss_value = losses.item()

        loss_hist.send(loss_value)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 50 == 0:
            print(f"Iteration #{itr} loss: {loss_value}")

        itr += 1
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}")  
    evaluate(model, val_dataloader, device=device)

Save the training weights

In [ ]:
torch.save(model.state_dict(), 'fasterrcnn_resnet50_fpn.pth')

Loading the saved weights

In [ ]:
model.load_state_dict(torch.load("../input/fasterrcnn-weights/fasterrcnn_resnet50_fpn_best.pth"))

Now let's test the model and see the results

In [ ]:
images, targets = next(iter(val_dataloader))
images = list(img.to(DEVICE) for img in images)
print(images[0].shape)
targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
boxes = targets[1]['boxes'].cpu().numpy().astype(np.int32)
sample = images[1].permute(1,2,0).cpu().numpy()

In [ ]:
model.eval()
cpu_device = torch.device("cpu")
# print(images[0].shape)
outputs = model(images)
outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

In [ ]:
test_imgs_paths = pd.read_csv(os.path.join(DATAPATH,'sample_submission.csv'))
test_img_paths = test_imgs_paths.image_id.values
test_dir = DATAPATH+"/test"

In [ ]:
predictions=[]
image_ids=[]
with torch.no_grad():
    for path in test_img_paths:
        img_path = os.path.join(test_dir,path+".jpg")
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        image = np.transpose(image,(2,0,1))
    #     print(image.shape)
        image = torch.tensor(image,dtype=torch.float)
        image = image.unsqueeze(0)
    #     print(image.shape)
        image = image.to(DEVICE)
        outputs = model(image)

        predict=[]
        outputs = outputs[0]
        for i in range(len(outputs['boxes'])):
            temp = np.array([str(outputs['scores'][i].item()),str(outputs['boxes'][i][0].item()),str(outputs['boxes'][i][1].item()),str(outputs['boxes'][i][2].item()-outputs['boxes'][i][0].item()),str(outputs['boxes'][i][3].item()-outputs['boxes'][i][1].item())])
            predict.append(temp)
        predict = np.array(predict).flatten()
        predict = ' '.join(predict.flatten())
        image_ids.append(path)
        predictions.append(predict)
    print("------------Generating Submission File---------")
    df = pd.DataFrame({"image_id":image_ids,"PredictionString":predictions})
    df.to_csv('./submission.csv.gz',index=False,compression='gzip')

We will use Intersection Over Union (IOU) metric as a benchmark.

In [ ]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return interArea, float(boxAArea + boxBArea - interArea)

Now, we try the model on the validation set, and see the result. We also calculate the IOU of each prediction.

In [ ]:
images, targets = next(iter(val_dataloader))
images = list(img.to(DEVICE) for img in images)
#print(images[0].shape)
targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]
boxes = targets[1]
#sample = images[1].permute(1,2,0).cpu().numpy()

model.eval()
cpu_device = torch.device("cpu")
# print(images[0].shape)
outputs = model(images)
outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
#outputs = outputs

val_iou = []

for i in range(len(outputs)):
    b1 = targets[i]['boxes'].cpu().numpy().astype(np.int32)
    b2 = outputs[i]['boxes'].detach().numpy().astype(np.int32)
    sample = images[i].permute(1,2,0).cpu().numpy()
    
    scores = outputs[i]['scores'].detach().numpy()
   # print(scores)
    #b2 = b2[scores >= 0.5].astype(np.int32)
    #print(b1.shape)
    #print(b1)
    
    b2 = [x for _ , x in sorted(zip(scores,b2), key=lambda x: x[0], reverse = True)]
    b2 = b2[:(b1.shape[0])]
    b_good = []
    #print(len(b2))
    numerator = 0
    denominator = 0
    
    for bb2 in b2:
        for bb1 in b1:
            #print(bb1, bb2)
            o = bb_intersection_over_union(bb1, bb2)
           # print(o)
            num = o[0]
            dem = o[1]
            if num>0 and num/dem>0.5:
                numerator += num
                denominator +=dem
                b_good.append(bb2)
    iou = numerator/denominator
    print(iou)
    fig, ax = plt.subplots(1, 1, figsize=(16, 8))

    for box in b1:
        cv2.rectangle(sample,
                      (box[0], box[1]),
                      (box[2], box[3]),
                      (220, 0, 0), 3)
    for box in b_good:
        cv2.rectangle(sample,
                      (box[0], box[1]),
                      (box[2], box[3]),
                      (0, 0, 220), 3)
    ax.set_axis_off()
    ax.imshow(sample)
    
    
    
print(val_iou)